In [ ]:
import sqlite3
import pandas as pd
import numpy as np
from datetime import timedelta
import matplotlib.pyplot as plt
import seaborn as sns

# Connect to the SQLite database
conn = sqlite3.connect('room_data.db') 

## NAIA anomalies

In [ ]:
# Load anomalies from the NAIA model
anomalies_NAIA_df = pd.read_sql_query("""
    SELECT * FROM anomalies
    WHERE source_model = 'NAIA' AND timestamp >= datetime('now', '-180 day')
""", conn, parse_dates=['timestamp'])

conn.close()

anomalies_NAIA_df['hour'] = anomalies_NAIA_df['timestamp'].dt.floor('h')

anomalies_NAIA_df['date'] = anomalies_NAIA_df['hour'].dt.date
last_date = anomalies_NAIA_df['date'].max()

last_day_df = anomalies_NAIA_df[anomalies_NAIA_df['date'] == last_date]
hourly_counts_last_day = last_day_df.drop_duplicates(subset=['room_id', 'hour']) \
    .groupby('room_id').size().reset_index(name='anomalous_hours')


plt.figure(figsize=(6, 4))
sns.barplot(data=hourly_counts_last_day, x='room_id', y='anomalous_hours', palette='Set2')
plt.title(f'Anomalous Hours per Room on {last_date}')
plt.xlabel('Room ID')
plt.ylabel('Anomalous Hours')
plt.tight_layout()
plt.show()

print(f"Anomalous Hours per Room on {last_date}:")
print(hourly_counts_last_day.to_string(index=False))


hourly_counts_overall = anomalies_NAIA_df.drop_duplicates(subset=['room_id', 'hour']) \
    .groupby('room_id').size().reset_index(name='anomalous_hours')

plt.figure(figsize=(12, 5))
sns.barplot(data=hourly_counts_overall, x='room_id', y='anomalous_hours', palette='crest')
plt.title('Total Distinct Anomalous Hours per Room (NAIA Model)')
plt.xlabel('Room ID')
plt.ylabel('Anomalous Hours')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

## LSTM_AE anomalies

In [ ]:
conn = sqlite3.connect("room_data.db")
anomalies_LSTMAE_df = pd.read_sql_query("""
    SELECT CAST(room_id AS INTEGER) AS room_id,
           timestamp
    FROM anomalies
    WHERE source_model = 'LSTM-AE' AND timestamp >= datetime('now', '-180 day')
""", conn, parse_dates=['timestamp'])
conn.close()

# Drop any bogus room_id 0
anomalies_LSTMAE_df = anomalies_LSTMAE_df[anomalies_LSTMAE_df['room_id'] != 0]

if anomalies_LSTMAE_df['timestamp'].dtype == object:
    anomalies_LSTMAE_df['timestamp'] = (
        anomalies_LSTMAE_df['timestamp']
        .str.replace('T', ' ', regex=False)
        .str.replace('Z', '',  regex=False)
        .pipe(pd.to_datetime, errors='coerce')
    )
anomalies_LSTMAE_df = anomalies_LSTMAE_df.dropna(subset=['timestamp'])

anomalies_LSTMAE_df['hour'] = anomalies_LSTMAE_df['timestamp'].dt.floor('h')

anomalies_LSTMAE_df['date'] = anomalies_LSTMAE_df['hour'].dt.date
last_date = anomalies_LSTMAE_df['date'].max()

last_day = anomalies_LSTMAE_df[anomalies_LSTMAE_df['date'] == last_date]
hourly_counts_last_day = (
    last_day
    .drop_duplicates(subset=['room_id', 'hour'])
    .groupby('room_id')
    .size()
    .reset_index(name='anomalous_hours')
)

plt.figure(figsize=(6, 4))
sns.barplot(
    data=hourly_counts_last_day,
    x='room_id', y='anomalous_hours',
    palette='Set2'
)
plt.title(f'Anomalous Hours per Room on {last_date}')
plt.xlabel('Room ID')
plt.ylabel('Anomalous Hours')
plt.tight_layout()
plt.show()

print(f"\n Anomalous Hours per Room on {last_date}:")
print(hourly_counts_last_day.to_string(index=False))

hourly_counts_overall = (
    anomalies_LSTMAE_df
    .drop_duplicates(subset=['room_id', 'hour'])
    .groupby('room_id')
    .size()
    .reset_index(name='anomalous_hours')
)

plt.figure(figsize=(12, 5))
sns.barplot(
    data=hourly_counts_overall,
    x='room_id', y='anomalous_hours',
    palette='crest'
)
plt.title('Total Distinct Anomalous Hours per Room (LSTM-AE Model)')
plt.xlabel('Room ID')
plt.ylabel('Anomalous Hours')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

## Anomalous days detected by both/either models

In [ ]:
def load_and_process(model_name):
    conn = sqlite3.connect('room_data.db')
    df = pd.read_sql_query(
        f"""
        SELECT CAST(room_id AS INTEGER) AS room_id,
               timestamp
        FROM anomalies
        WHERE source_model = '{model_name}'
        """,
        conn,
        parse_dates=['timestamp']
    )
    conn.close()

    # Drop any bogus room_id 0
    df = df[df['room_id'] != 0]

    if df['timestamp'].dtype == object:
        df['timestamp'] = (
            df['timestamp']
              .str.replace('T', ' ', regex=False)
              .str.replace('Z', '', regex=False)
              .pipe(pd.to_datetime, errors='coerce')
        )
    df = df.dropna(subset=['timestamp'])

    df['day'] = df['timestamp'].dt.floor('D')
    return df

naia_df = load_and_process('NAIA')
lstm_df = load_and_process('LSTM-AE')

naia_unique = naia_df.drop_duplicates(subset=['room_id', 'day'])
lstm_unique = lstm_df.drop_duplicates(subset=['room_id', 'day'])

common_days = pd.merge(
    naia_unique[['room_id', 'day']],
    lstm_unique[['room_id', 'day']],
    on=['room_id', 'day'],
    how='inner'
)

print("Anomalous days detected by both NAIA and LSTM-AE models (room_id, day):")
print(common_days.sort_values(['room_id', 'day']).to_string(index=False))

counts = (
    common_days
    .groupby('room_id')['day']
    .nunique()
    .reset_index(name='anomalous_day_count')
    .sort_values('room_id')
)

plt.figure(figsize=(10, 6))
plt.bar(counts['room_id'].astype(str), counts['anomalous_day_count'])
plt.xlabel('Room ID')
plt.ylabel('Number of Common Anomalous Days')
plt.title('Days Flagged by Both NAIA and LSTM-AE Models per Room')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

all_anoms = pd.concat([naia_df, lstm_df], ignore_index=True)

all_anoms['hour'] = all_anoms['timestamp'].dt.floor('H')

all_unique_hours = all_anoms.drop_duplicates(subset=['room_id', 'hour'])

hour_counts = (
    all_unique_hours
    .groupby('room_id')['hour']
    .nunique()
    .reset_index(name='anomalous_hour_count')
    .sort_values('room_id')
)

plt.figure(figsize=(10, 6))
plt.bar(hour_counts['room_id'].astype(str), hour_counts['anomalous_hour_count'])
plt.xlabel('Room ID')
plt.ylabel('Number of Anomalous Hours')
plt.title('Anomalous Hours Flagged by Either NAIA or LSTM-AE per Room')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
import sqlite3
import statistics

# Room IDs to check
room_ids = [
    1, 2, 3, 4, 6, 7, 8, 9, 20, 21, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37,
    38, 39, 40, 41, 311, 51, 56, 57, 59, 60, 87, 88, 89, 90, 91, 92, 93, 94,
    95, 104, 105, 115, 116, 117, 297, 261,
]

db_path = "room_data.db"

counts = [] 

with sqlite3.connect(db_path) as conn:
    cur = conn.cursor()
    for rid in room_ids:
        cur.execute(
            "SELECT COUNT(*) FROM sensor_data_history WHERE room_id=?", (rid,)
        )
        count = cur.fetchone()[0]
        counts.append((rid, count))
        print(f"Room {rid}: {count}")

if counts:
    totals = [c for _, c in counts]
    total = sum(totals)
    average = total / len(counts)
    median_val = statistics.median(totals)
    max_room, max_val = max(counts, key=lambda x: x[1])
    min_room, min_val = min(counts, key=lambda x: x[1])

    print("\nSummary:")
    print(f"Total rows: {total}")
    print(f"Average rows per room: {average:.2f}")
    print(f"Median rows per room: {median_val}")
    print(f"Max rows: {max_val} (room {max_room})")
    print(f"Min rows: {min_val} (room {min_room})")
else:
    print("No data found for the given rooms.")
